In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import re
import time

In [2]:
# Create DataFrame from CSV file
nlp_df = pd.read_csv('../Data/nlp_df.csv')
nlp_df = nlp_df.drop(['non_alpha_words', 'words'], axis=1)
nlp_df.head(3)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,...,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,...,0.8330,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,"['', 'im', 'like', 'water', 'ship', 'rolled', ...","(1, 'en')"
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,...,0.0805,0.00000,0.366,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,"['', 'didnt', 'even', 'wanna', 'go', 'whyd', '...","(1, 'en')"
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,...,0.2130,0.00000,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,"['', 'cut', 'piece', 'bleed', 'internally', 'l...","(1, 'en')"


In [3]:
# Create a list of all words, word counts, unique word counts, and filtered words
t0 = time.time()
words_list = []
word_counts = []
unique_word_counts = []
filtered_words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    filtered_words = filtered_words.replace(',', '').replace("'", '')
    filtered_words = filtered_words.replace('[', '').replace(']', '')
    filtered_words = filtered_words.replace('#', '').replace('&nbsp', '')
    filtered_words = filtered_words.replace('?', '? ').replace(',', '')
    filtered_words = filtered_words.replace('/', ' ')
    filtered_words = filtered_words.replace('\\u200a', '').replace('\\u200b', '')
    filtered_words = filtered_words.replace('\\u2063', '').replace('\u202f', '')
    filtered_words = filtered_words.replace('\\u2028', ' ').replace('\\u2008', ' ')
    while ('\\u200e' in filtered_words) or ('\\xa0' in filtered_words):
        filtered_words = filtered_words.replace('\\u200e', '')
        filtered_words = filtered_words.replace('\\xa0', '')
    filtered_words_list.append(filtered_words)
    words = filtered_words.strip().split(' ')
    word_counts.append(len(words))
    unique_words = list(set(words))
    unique_word_counts.append(len(unique_words))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(word_columns)

Run time: 2.0935609340667725 seconds


39244

In [4]:
# Add word_count and unique_word_count columns and replace the filtered column
nlp_df['word_count'] = word_counts
nlp_df['unique_word_count'] = unique_word_counts
nlp_df['filtered'] = filtered_words_list
nlp_df.head(1)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,...,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,filtered,language,word_count,unique_word_count
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,...,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,im like water ship rolled night rough surface...,"(1, 'en')",227,87


In [5]:
# Remove songs with less than 25 unique words
nlp_df = nlp_df[nlp_df['unique_word_count']>=25]
len(nlp_df)

8115

In [6]:
# Update word list
words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    unique_words = list(set(filtered_words.strip().split(' ')))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
len(word_columns)

39099

# Try to remove non English songs

In [7]:
# Find songs indeces with one language that is not English
one_lang_not_en = []
mult_langs_not_en = []
for index, row in nlp_df.iterrows():
    if (row['language'][1] == '1') & (row['language'][5:7] != 'en'):
        one_lang_not_en.append(index)
    if (row['language'][1] != '1') & (row['language'][5:7] != 'en'):
        mult_langs_not_en.append(index)
print(len(one_lang_not_en))
len(mult_langs_not_en)

99


103

In [8]:
# Remove non English songs
songs_to_keep = [78, 452, 715, 878, 1633, 2150, 2402, 2407, 2427, 2526, 2835, 2979, 3232,
                 3359, 3791, 4368, 4643, 5692, 6328, 6379, 6387, 6523, 6722, 7853]
songs_to_remove = [264, 924, 1610, 1618, 2299, 2368, 3688, 5482, 5656, 5658, 5688, 5695, 
                   5710, 5715, 5719, 5720, 5750, 5792, 6385]
songs_to_remove.extend([item for item in one_lang_not_en if item not in songs_to_keep])
nlp_df = nlp_df.drop(songs_to_remove)
print(len(songs_to_remove))
len(nlp_df)

104


8012

# Examine word counts by genre

In [9]:
# Remove the language column
nlp_df = nlp_df.drop('language', axis=1)

In [10]:
# Drop songs with incorrect lyrics from the DataFrame
nlp_df = nlp_df[(nlp_df['unique_word_count']!=3878) & (nlp_df['unique_word_count']!=1153)]
nlp_df = nlp_df[(nlp_df['unique_word_count']!=1000) & (nlp_df['unique_word_count']!=880)]
nlp_df = nlp_df[(nlp_df['unique_word_count']!=842) & (nlp_df['unique_word_count']!=607)]
nlp_df = nlp_df[(nlp_df['unique_word_count']!=569) & (nlp_df['word_count']!=1383)]
#nlp_df = nlp_df.drop([4242, 5672, 5666, 2443, 2460, 3510, 1374, 3539, 1237])
len(nlp_df)

8002

In [11]:
nlp_df['popularity'].describe()

count    7999.000000
mean       47.210776
std        22.694544
min         0.000000
25%        37.000000
50%        52.000000
75%        63.000000
max        99.000000
Name: popularity, dtype: float64

In [12]:
# Print the distributions of word counts for each quartile
word_distributions = {}
quartile1_unique_words = nlp_df[nlp_df['popularity']<=37]['word_count'].describe()
word_distributions['first_quartile'] = quartile1_unique_words

quartile2_unique_words = nlp_df[(nlp_df['popularity']>37) & (nlp_df['popularity']<=52)]['word_count'].describe()
word_distributions['second_quartile'] = quartile2_unique_words

quartile3_unique_words = nlp_df[(nlp_df['popularity']>52) & (nlp_df['popularity']<63)]['word_count'].describe()
word_distributions['third_quartile'] = quartile3_unique_words

quartile4_unique_words = nlp_df[nlp_df['popularity']>=63]['word_count'].describe()
word_distributions['fourth_quartile'] = quartile4_unique_words
word_distributions

{'first_quartile': count    2052.000000
 mean      154.047271
 std        77.705555
 min        26.000000
 25%       102.000000
 50%       140.000000
 75%       189.000000
 max       694.000000
 Name: word_count, dtype: float64,
 'second_quartile': count    2115.000000
 mean      163.676596
 std        87.487074
 min        36.000000
 25%       107.000000
 50%       144.000000
 75%       194.000000
 max       823.000000
 Name: word_count, dtype: float64,
 'third_quartile': count    1696.000000
 mean      191.904481
 std       104.901888
 min        32.000000
 25%       125.000000
 50%       169.000000
 75%       223.000000
 max       832.000000
 Name: word_count, dtype: float64,
 'fourth_quartile': count    2136.000000
 mean      205.630618
 std        98.102693
 min        33.000000
 25%       143.000000
 50%       185.000000
 75%       239.000000
 max       909.000000
 Name: word_count, dtype: float64}

In [13]:
# Create a dictionary of mean word distributions
categories = list(word_distributions.keys())
means = []
for category in word_distributions:
    means.append(word_distributions[category]['mean'])
mean_unique_word_counts = dict(zip(categories, means))
mean_unique_word_counts = dict(sorted(mean_unique_word_counts.items(), key=lambda item: item[1], reverse=True))
mean_unique_word_counts

{'fourth_quartile': 205.6306179775281,
 'third_quartile': 191.90448113207546,
 'second_quartile': 163.67659574468084,
 'first_quartile': 154.0472709551657}

In [14]:
# Find songs with the highest and lowest word counts
print('Song with the most words:')
display(nlp_df[nlp_df['word_count']==max(nlp_df['word_count'])][['song', 'artist', 'category', 'popularity', 'lyrics', 'word_count']])

print('Song with the least words:')
display(nlp_df[nlp_df['word_count']==min(nlp_df['word_count'])][['song', 'artist', 'category', 'popularity', 'lyrics', 'word_count']])


Song with the most words:


,song,artist,category,popularity,lyrics,word_count
1358,Rap God,Eminem,hiphop,64,Look I was gonna go easy on you not to hurt y...,909


Song with the least words:


,song,artist,category,popularity,lyrics,word_count
2581,Golf Hill Drive,Boys Life,rock,25,Stuck in this life thats not my own And whoeve...,26


In [15]:
# Create DataFrame of word count distributions
ranges = []
for category in word_distributions:
    ranges.append(word_distributions[category]['max'] - word_distributions[category]['min'])

columns = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'range']
df_rows = []
for index, category in enumerate(word_distributions):
    distribution = list(word_distributions[category])
    distribution.extend([ranges[index]])
    df_rows.append(distribution)
word_count_df = pd.DataFrame(df_rows, index=list(word_distributions.keys()), columns=columns)
word_count_df

,count,mean,std,min,25%,50%,75%,max,range
first_quartile,2052.0,154.047271,77.705555,26.0,102.0,140.0,189.0,694.0,668.0
second_quartile,2115.0,163.676596,87.487074,36.0,107.0,144.0,194.0,823.0,787.0
third_quartile,1696.0,191.904481,104.901888,32.0,125.0,169.0,223.0,832.0,800.0
fourth_quartile,2136.0,205.630618,98.102693,33.0,143.0,185.0,239.0,909.0,876.0


# Conclusions about word distributions by popularity
- The songs with the lowest 25% popularities have the lowest mean word count of 154 words
- The songs in the second quartile of popularity have a mean word count of 164 words
- The songs in the third quartile of popularity have a mean word count of 191 words
- The songs with the highest 25% popularities ahve the highest mean word count fo 206 words
- The song with the highest word count has a popularity of 64:
    - Song name: Rap God 
    - Artist: Eminem
    - Word count: 909 words
- The song with the lowest word count has a popularity of 25:
    - Song name: Golf Hill Drive 
    - Artist: Boys Life
    - Word count: 26 words

# Examine unique word counts by genre

In [16]:
# Print the distributions of unique word counts for each quartile
unique_word_distributions = {}
quartile1_unique_words = nlp_df[nlp_df['popularity']<=37]['unique_word_count'].describe()
unique_word_distributions['first_quartile'] = quartile1_unique_words

quartile2_unique_words = nlp_df[(nlp_df['popularity']>37) & (nlp_df['popularity']<=52)]['unique_word_count'].describe()
unique_word_distributions['second_quartile'] = quartile2_unique_words

quartile3_unique_words = nlp_df[(nlp_df['popularity']>52) & (nlp_df['popularity']<63)]['unique_word_count'].describe()
unique_word_distributions['third_quartile'] = quartile3_unique_words

quartile4_unique_words = nlp_df[nlp_df['popularity']>=63]['unique_word_count'].describe()
unique_word_distributions['fourth_quartile'] = quartile4_unique_words
unique_word_distributions

{'first_quartile': count    2052.000000
 mean       69.077485
 std        38.100541
 min        25.000000
 25%        47.000000
 50%        62.000000
 75%        79.000000
 max       486.000000
 Name: unique_word_count, dtype: float64,
 'second_quartile': count    2115.000000
 mean       73.972104
 std        44.322021
 min        25.000000
 25%        49.000000
 50%        63.000000
 75%        82.000000
 max       503.000000
 Name: unique_word_count, dtype: float64,
 'third_quartile': count    1696.000000
 mean       82.753538
 std        52.955756
 min        25.000000
 25%        52.750000
 50%        68.000000
 75%        92.000000
 max       544.000000
 Name: unique_word_count, dtype: float64,
 'fourth_quartile': count    2136.000000
 mean       82.632959
 std        47.983986
 min        25.000000
 25%        55.000000
 50%        70.000000
 75%        92.000000
 max       539.000000
 Name: unique_word_count, dtype: float64}

In [17]:
# Create a dictionary of mean unique word distributions
categories = list(word_distributions.keys())
means = []
for category in word_distributions:
    means.append(word_distributions[category]['mean'])
mean_unique_word_counts = dict(zip(categories, means))
mean_unique_word_counts = dict(sorted(mean_unique_word_counts.items(), key=lambda item: item[1], reverse=True))
mean_unique_word_counts

{'fourth_quartile': 205.6306179775281,
 'third_quartile': 191.90448113207546,
 'second_quartile': 163.67659574468084,
 'first_quartile': 154.0472709551657}

In [22]:
# Find songs with the highest and lowest  word counts
print('Song with the most unique words:')
display(nlp_df[nlp_df['unique_word_count']==max(nlp_df['unique_word_count'])][['song', 'artist', 'category', 'popularity', 'lyrics', 'unique_word_count']])

print('Popularities of songs that have the minimum unique word count of 25 words:')
print(set(nlp_df[nlp_df['unique_word_count']==min(nlp_df['unique_word_count'])]['popularity']), '\n')

print(nlp_df[nlp_df['unique_word_count']==min(nlp_df['unique_word_count'])]['popularity'].describe())

min_word_count_df = nlp_df[nlp_df['unique_word_count']==min(nlp_df['unique_word_count'])]
print(f'{len(min_word_count_df)} songs have the minimum unique word count of 25 words.')



Song with the most unique words:


,song,artist,category,popularity,lyrics,unique_word_count
1268,Momentum (feat. Black Thought & Benny The Butc...,Russ,hiphop,61,Yeah yeah Dont compare me to employees who ...,544


Popularities of songs that have the minimum unique word count of 25 words:
{0, 4, 25, 26, 30, 31, 35, 38, 42, 43, 45, 46, 48, 49, 60, 61, 62, 70, 84} 

count    25.000000
mean     36.160000
std      24.463715
min       0.000000
25%      25.000000
50%      42.000000
75%      49.000000
max      84.000000
Name: popularity, dtype: float64
25 songs have the minimum unique word count of 25 words.


In [19]:
# Create DataFrame of unique word count distributions
ranges = []
for category in unique_word_distributions:
    ranges.append(unique_word_distributions[category]['max'] - unique_word_distributions[category]['min'])

columns = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'range']
df_rows = []
for index, category in enumerate(unique_word_distributions):
    distribution = list(unique_word_distributions[category])
    distribution.extend([ranges[index]])
    df_rows.append(distribution)
word_count_df = pd.DataFrame(df_rows, index=list(unique_word_distributions.keys()), columns=columns)
word_count_df

,count,mean,std,min,25%,50%,75%,max,range
first_quartile,2052.0,69.077485,38.100541,25.0,47.00,62.0,79.0,486.0,461.0
second_quartile,2115.0,73.972104,44.322021,25.0,49.00,63.0,82.0,503.0,478.0
third_quartile,1696.0,82.753538,52.955756,25.0,52.75,68.0,92.0,544.0,519.0
fourth_quartile,2136.0,82.632959,47.983986,25.0,55.00,70.0,92.0,539.0,514.0


# Conclusions about unique word distributions by popularity
- The songs with the lowest 25% popularities have the lowest mean unique word count of 69 words
- The song with the highest unique word count has a popularity of 61:
    - Song name: Momentum 
    - Artist: Russ
    - Unique word count: 544 words
- The average popularity of the 25 songs with minimum unique word count was 36
    - The highest popularity of the songs with the minimum unique word count was 84
    - The lowest popularity of the songs with the minimum unique word count was 0